<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## Load Data

In [ ]:
train_csv = pd.read_csv(
    "data/autass_data2.csv",
    header=None,
    dtype=np.double,
)
data = np.array(train_csv.values[:, 1:50])
del train_csv

In [ ]:
X = data[:, 0:48]
y = data[:, 48].astype(int) - 1

yt = copy.copy(y)
yt[yt == 0] = 20
yt[yt == 1] = 21
yt[yt == 2] = 22
yt[yt == 3] = 23
yt[yt == 4] = 26
yt[yt == 5] = 24
yt[yt == 6] = 27
yt[yt == 7] = 29
yt[yt == 8] = 30
yt[yt == 9] = 25
yt[yt == 10] = 28
yt -= 20
y = yt
del yt

## Config

In [ ]:
epochs = 50
batch_size = 538
lr = 1

## Single Layer

### MLMVN [48-100-11]

In [ ]:
PATH = str(Path.cwd() / "models/autass-mlmvn_48-100-11.pt")

In [ ]:
class Model(nn.Module):
    def __init__(self, categories, periodicity):
        super().__init__()
        self.categories = categories
        self.periodicity = periodicity
        self.first_linear = FirstLayer(48, 100)
        self.phase_act1 = cmplx_phase_activation()
        self.linear_out = OutputLayer(100, 11)
        self.phase_act2 = cmplx_phase_activation()
        # Hooks
        self.first_layer_hook_handle = self.first_linear.register_full_backward_hook(
            self.first_layer_backward_hook
        )
        self.output_hook_handle = self.linear_out.register_full_backward_hook(
            self.output_layer_backward_hook
        )

    def forward(self, x):
        x = self.first_linear(x)
        x = self.phase_act1(x)
        x = self.linear_out(x)
        x = self.phase_act2(x)
        return x

    def first_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("first_layer", module, grad_input, grad_output)

    def hidden_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("hidden_layer", module, grad_input, grad_output)

    def output_layer_backward_hook(self, module, grad_input, grad_output):
        fc_hook("output_layer", module, grad_input, grad_output)

    def angle2class(self, x: torch.tensor) -> torch.tensor:
        tmp = x.angle() + 2 * np.pi
        angle = torch.remainder(tmp, 2 * np.pi)

        # This will be the discrete output (the number of sector)
        o = torch.floor(self.categories * self.periodicity * angle / (2 * np.pi))
        return torch.remainder(o, self.categories)

    def predict(self, x):
        """
        Performs the prediction task of the network

        Args:
          x: torch.Tensor
            Input tensor of size ([3])

        Returns:
          Most likely class i.e., Label with the highest score
        """
        # Pass the data through the networks
        output = self.forward(x)

        # # Choose the label with the highest score
        # return torch.argmax(output, 1)
        return self.angle2class(output)

    def initialize_weights(self, initilizer="uniform"):
        if initilizer == "uniform":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_uniform_independent_(m.weights, -0.5, 0.5)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_uniform_independent_(m.weights, -0.5, 0.5)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_uniform_independent_(m.weights, -0.5, 0.5)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "normal":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_normal_independent_(
                        m.weights,
                    )
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_normal_independent_(
                        m.weights,
                    )
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_normal_independent_(
                        m.weights,
                    )
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "ones":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.ones_(m.weights, imag_zero=True)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.ones_(m.weights, imag_zero=True)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.ones_(m.weights, imag_zero=True)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "zeros":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.zeros_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.zeros_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.zeros_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "kaiming_normal":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_kaiming_normal_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_kaiming_normal_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_kaiming_normal_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "kaiming_uniform":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_kaiming_uniform_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_kaiming_uniform_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_kaiming_uniform_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "xavier_normal":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_xavier_normal_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_xavier_normal_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_xavier_normal_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "xavier_uniform":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_xavier_uniform_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_xavier_uniform_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_xavier_uniform_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "trabelsi_standard_glorot":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "trabelsi_independent_glorot":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "trabelsi_standard_xavier":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights, kind="xavier")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights, kind="xavier")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights, kind="xavier")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "trabelsi_independent_xavier":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights, kind="xavier")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights, kind="xavier")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights, kind="xavier")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "trabelsi_standard_kaiming":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights, kind="kaiming")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights, kind="kaiming")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights, kind="kaiming")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "trabelsi_independent_kaiming":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights, kind="kaiming")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights, kind="kaiming")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights, kind="kaiming")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "trabelsi_standard_he":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights, kind="he")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights, kind="he")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_trabelsi_standard_(m.weights, kind="he")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "trabelsi_independent_he":
            for m in self.modules():
                if isinstance(m, FirstLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights, kind="he")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, HiddenLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights, kind="he")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, OutputLayer):
                    cmplx_init.cplx_trabelsi_independent_(m.weights, kind="he")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
        elif initilizer == "standard":
            pass


def fit(model, X, y, epochs, batch_size, optimizer, criterion, categories, periodicity):
    # List of losses for visualization
    losses = []
    scores = []
    acc_best = 0

    for i in range(epochs):
        # Pass the data through the network and compute the loss
        # We'll use the whole dataset during the training instead of using batches
        # in to order to keep the code simple for now.

        batch_loss = []

        for j in range((X.shape[0] - 1) // batch_size + 1):
            start_j = j * batch_size
            end_j = start_j + batch_size
            xb = X[start_j:end_j]
            yb = y[start_j:end_j]

            y_pred = model(xb)
            loss = criterion(y_pred, yb, categories, periodicity)
            batch_loss.append((torch.abs(loss)).detach().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step(inputs=xb, layers=list(model.children()))

        losses.append(sum(batch_loss) / len(batch_loss))
        if i % 10 == 9:
            print(f"Epoch {i} loss is {losses[-1]}")
        y_pred = model.predict(X)
        scores.append(accuracy(y_pred.squeeze(), y))

        Logger.current_logger().report_scalar(
            "Loss/Acc", "Loss", iteration=i, value=losses[-1]
        )
        writer.add_scalar("Loss", losses[-1], i)
        Logger.current_logger().report_scalar(
            "Loss/Acc", "Acc", iteration=i, value=scores[-1]
        )
        writer.add_scalar("Accuracy", scores[-1], i)

        for key in model_dict:
            for key_layer in model_dict[key]:
                if key_layer in ["weights", "bias"]:
                    log_label = str(key) + "_" + str(key_layer)
                    log_label.replace(" ", "")
                    writer.add_histogram(
                        log_label + "_real", model_dict[key][key_layer].real, i
                    )
                    writer.add_histogram(
                        log_label + "_imag", model_dict[key][key_layer].imag, i
                    )
                    writer.add_histogram(
                        log_label + "_mag", torch.abs(model_dict[key][key_layer]), i
                    )
                    writer.add_histogram(
                        log_label + "_angle", torch.angle(model_dict[key][key_layer]), i
                    )

        # writer.add_histogram("distribution centers", x + n_iter, i)
        if scores[-1] > acc_best:
            acc_best = scores[-1]
            torch.save(model.state_dict(), PATH)

    writer.close()
    return losses, scores

### Init

In [ ]:
initilizers = [
    "uniform",
    "normal",
    # "zeros",
    # "ones",
    "kaiming_normal",
    "kaiming_uniform",
    "xavier_normal",
    "xavier_uniform",
    "trabelsi_standard_glorot",
    "trabelsi_independent_glorot",
    "trabelsi_standard_xavier",
    "trabelsi_independent_xavier",
    "trabelsi_standard_kaiming",
    "trabelsi_independent_kaiming",
    "trabelsi_standard_he",
    "trabelsi_independent_he",
    "standard",
]

In [ ]:
for initilizer in initilizers:
    model = Model(categories=categories, periodicity=periodicity)
    model.initialize_weights(initilizer=initilizer)
    criterion = ComplexMSELoss.apply
    optimizer = ECL(model.parameters(), lr=lr)

    task = Task.init(
        project_name="mlmvn",
        task_name="SDD-mlmvn-[48-100-11]",
        tags=["mlmvn", "SDD", "initilizer"],
    )
    writer = SummaryWriter()

    #  capture a dictionary of hyperparameters with config
    config_dict = {
        "learning_rate": lr,
        "epochs": epochs,
        "batch_size": batch_size,
        "optim": "ECL",
        "categories": categories,
        "periodicity": periodicity,
        "layer": "[48-100-11]",
        "initilizer": initilizer,
    }
    task.connect(config_dict)

    x_train, x_valid, y_train, y_valid = get_splitted_data(X, y, neuronCats)

    losses, scores = fit(
        model,
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        optimizer=optimizer,
        criterion=criterion,
        categories=categories,
        periodicity=periodicity,
    )

    model.load_state_dict(torch.load(PATH))

    y_pred = model.predict(x_train)
    acc = accuracy(y_pred.squeeze(), y_train)
    print("Train Acc.: ", acc)
    Logger.current_logger().report_single_value(
        name="Train Acc.",
        value=acc,
    )

    y_pred = model.predict(x_valid)
    acc = accuracy(y_pred.squeeze(), y_valid)
    print("Val Acc.: ", acc)
    Logger.current_logger().report_single_value(
        name="Val Acc.",
        value=acc,
    )
    print(classification_report(y_valid, y_pred.detach().numpy(), zero_division=0))

    task.mark_completed()
    task.close()

ClearML Task: created new task id=44d8ea5cbf604878b92f402cc8b0eddc
ClearML results page: http://194.94.231.172:8080/projects/cdefd6ee85454e49be01962ad715eca0/experiments/44d8ea5cbf604878b92f402cc8b0eddc/output/log


/tmp/ipykernel_20526/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



2022-09-20 19:22:00,485 - clearml.frameworks - INFO - Found existing registered model id=0f73e6db01fc42988672e4f44c0add5f [/home/antonpfeifer/Documents/mlmvn/nbs/examples/autass/models/autass-mlmvn_48-100-11.pt] reusing it.
Epoch 9 loss is 0.12282228621301199
Epoch 19 loss is 0.09224345408146964
Epoch 29 loss is 0.10864849515390713
Epoch 39 loss is 0.0810078859787397
Epoch 49 loss is 0.0898406030411643
Train Acc.:  0.9418023330342263
Val Acc.:  0.9291634623600786
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      1074
           1       0.92      0.91      0.91      1089
           2       0.98      0.96      0.97      1044
           3       0.98      0.97      0.98      1048
           4       0.96      0.92      0.94      1057
           5       0.93      0.92      0.92      1072
           6       0.93      0.91      0.92      1066
           7       1.00      1.00      1.00      1103
           8       1.00      1.00      1.00  

/tmp/ipykernel_20526/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.6195961883261095
Epoch 19 loss is 0.4368921566730075
Epoch 29 loss is 0.26458930689298554
Epoch 39 loss is 0.18611469218135965
Epoch 49 loss is 0.1550662377159612
Train Acc.:  0.8358543776438918
Val Acc.:  0.8256002734341622
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1074
           1       0.85      0.76      0.80      1089
           2       0.93      0.79      0.86      1044
           3       0.93      0.92      0.92      1048
           4       0.91      0.84      0.88      1057
           5       0.83      0.80      0.82      1072
           6       0.90      0.86      0.88      1066
           7       0.98      0.98      0.98      1103
           8       1.00      1.00      1.00      1108
           9       0.88      0.88      0.88      1030
          10       0.90      0.80      0.84      1012

   micro avg       0.92      0.87      0.89     11703
   macro avg       0.92      0.87      0.89     11703

...

Epoch 9 loss is 0.1285782571954889
Epoch 19 loss is 0.09804718683267999
Epoch 29 loss is 0.07395459259074805
Epoch 39 loss is 0.060897612847309844
Epoch 49 loss is 0.07065708804235743
Train Acc.:  0.9491945477075588
Val Acc.:  0.9430060668204734
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      1074
           1       0.94      0.92      0.93      1089
           2       0.99      0.98      0.99      1044
           3       0.98      0.96      0.97      1048
           4       0.95      0.95      0.95      1057
           5       0.94      0.93      0.93      1072
           6       0.96      0.92      0.94      1066
           7       1.00      1.00      1.00      1103
           8       1.00      1.00      1.00      1108
           9       0.96      0.93      0.94      1030
          10       0.99      0.96      0.97      1012

   micro avg       0.97      0.96      0.96     11703
   macro avg       0.97      0.96      0.96     11

/tmp/ipykernel_20526/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.12129981536387262
Epoch 19 loss is 0.07506399494782308
Epoch 29 loss is 0.06446026524658817
Epoch 39 loss is 0.06043589024096862
Epoch 49 loss is 0.0524866021964434
Train Acc.:  0.9561594667350339
Val Acc.:  0.9452277193881911
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1074
           1       0.96      0.91      0.93      1089
           2       0.99      0.98      0.98      1044
           3       0.99      0.97      0.98      1048
           4       0.97      0.95      0.96      1057
           5       0.95      0.93      0.94      1072
           6       0.96      0.94      0.95      1066
           7       1.00      0.99      1.00      1103
           8       1.00      1.00      1.00      1108
           9       0.94      0.95      0.95      1030
          10       0.99      0.95      0.97      1012

   micro avg       0.98      0.96      0.97     11703
   macro avg       0.98      0.96      0.97     117

/tmp/ipykernel_20526/161459083.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 9 loss is 0.1182056404424841
Epoch 19 loss is 0.07147076034313422
Epoch 29 loss is 0.05620083264376806
Epoch 39 loss is 0.05679742307294903
